<a href="https://colab.research.google.com/github/TaroKawa/CFDxDL/blob/master/shock_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ToDo List
* よい初期値を探す（conv, convtransposed)
* distance functioniを作成する
* 損失関数を作成する(衝撃波部分＋オイラー方程式部分）
* 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import shutil

shutil.copytree("/content/drive/My Drive/shape_to_shock/", "/content/ss")

'/content/ss'

In [0]:
import os

In [0]:
os.chdir("ss")

In [5]:
!ls

check_sinbeta.py  data	foo.py	loader.py  main.py  __pycache__  shock_net.py


In [0]:

# -*- coding: utf-8 -*-
"""
Shcok Net
==============

**Author**: Taro Kawasaki

"""

from __future__ import print_function

import glob
import os.path as osp

import numpy as np
import torch.utils.data as data

# %matplotlib inline
import random

import matplotlib.animation as animation
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.utils as vutils
import torch.nn.functional as F

from IPython.display import HTML

# Data Loader

In [0]:


def make_data_path_list(phase="train"):
    """

    Parameters
    ----------
    phase : 'train' or 'val'

    Returns
    -------
    path_list : list
    """

    #rootpath = "./data/"
    rootpath = "./data/"
    target_path = osp.join(rootpath + phase + '/**/*/')
    #print(target_path)
    path_list = []

    for path in glob.glob(target_path):
        path_list.append(path)
    #print(path_list)
    return path_list


class Dataset(data.Dataset):
    """

    Attributes
    ----------
    file_list : list
    transform : object
    phase : 'train' or 'test'
    """

    def __init__(self, file_list, phase='train'):
        self.file_list = file_list  # file path
        # self.transform = transform  #
        self.phase = phase  # train or val
        self.size = 196

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        read_list = ["pressure", "mach","rho","temperature"]

        inputs = []
        outputs= []
        labels = []

        data_path = self.file_list[index]
        # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
        temp_label = data_path.split("/")[4].split("_")
        labels.append(float(temp_label[0][4:]))
        labels.append(float(temp_label[1][5:]))
        # = = = = = = = = = = = = = = = = = = = = = = = = = = = = = =
        delta_x=random.randrange(256-self.size)
        delta_y=random.randrange(256-self.size)
        for item in read_list:
            path = osp.join(data_path + "/" + item + ".csv")
            data = np.loadtxt(path, delimiter=",")
            data = data.T
            data=data[delta_y:self.size+delta_y,delta_x:self.size+delta_x]
            data = np.reshape(data, (self.size, self.size))
            if item=="pressure":
                shape_index=abs(data)<=1e-4
                inputs.append((shape_index.astype(int)))
                #plt.imshow(shape_index.astype(int),interpolation="nearest")
                #plt.show()
#
                #plt.imshow(data,interpolation="nearest")
                #plt.show()
            outputs.append(data)


        inputs=np.array(inputs,dtype="float64")
        outputs=np.array(outputs,dtype="float64")
        labels=np.array(labels,dtype="float64")

        return inputs, labels, outputs


def load(phase, batch_size):
    if phase == "train":
        train_dataset = Dataset(file_list=make_data_path_list("train"), phase="train")
        train_dataloader = data.DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True
        )
        return train_dataloader
        
    if phase == "val":
        train_dataset = Dataset(file_list=make_data_path_list("train"), phase="train")
        train_dataloader = data.DataLoader(
            train_dataset, batch_size=batch_size, shuffle=True
        )
        return train_dataloader



# HyperParameters


In [34]:
# Set random seed for reproducibility
manualSeed = 999
# manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

######################################################################
# Inputs
# ------

# Batch size during training
batch_size = 2

# input size
input_size=200

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = torch.cuda.device_count()
print("Let's use", ngpu, "GPUs!")

# Number of conv output channels
conv_channel1 = 4

# Number of conv output channels
conv_channel2 = 8

# Number of conv output channels
conv_channel3 = 16

# Number of hidden layer
n_hidden= 10

# Number of channels for output
output_channel= 4

######################################################################
# Data
# Create the dataloader

train_dataloader=load("train",batch_size=batch_size)

Random Seed:  999
Let's use 1 GPUs!


In [0]:
#while(True):
#    for i in train_dataloader:
#        print("")

In [0]:
######################################################################
# Implementation
# --------------
#
def weights_init(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif type(m)==nn.Conv2d:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif type(m)==nn.BatchNorm2d:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
    elif type(m)==nn.ConvTranspose2d:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


    #classname = m.__class__.__name__
    #if classname.find('Conv') != -1:
    #    nn.init.normal_(m.weight.data, 0.0, 0.02)
    #elif classname.find('BatchNorm') != -1:
    #    nn.init.normal_(m.weight.data, 1.0, 0.02)
    #    nn.init.constant_(m.bias.data, 0)


In [0]:
######################################################################
# Encoder
# ~~~~~~~~~
# Encoder Code
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder,self).__init__()
        self.channel =1
        self.model=nn.Sequential(
            # layer 1
            nn.Conv2d(self.channel, conv_channel1, kernel_size=4, stride=4, bias=True),
            nn.BatchNorm2d(conv_channel1),
            nn.LeakyReLU(0.1),

            # layer 2
            nn.Conv2d(conv_channel1, conv_channel2, kernel_size=4, stride=4, bias=True),
            nn.BatchNorm2d(conv_channel2),
            nn.LeakyReLU(0.1),

            # layer 3
            nn.Conv2d(conv_channel2, conv_channel3, kernel_size=4, stride=4, bias=True),
            nn.BatchNorm2d(conv_channel3),
            nn.LeakyReLU(0.1),

        )
    def forward(self,input):
        return self.model(input)

In [0]:
######################################################################
# concate
# ~~~~~~~~~
# Linear Code

class Linear(nn.Module):
    def __init__(self):
        super(Linear,self).__init__()
        self.n_hidden=n_hidden
        self.size=3
        self.model=nn.Sequential(
            # layer 1
            nn.Linear(conv_channel3*self.size**2+2,self.n_hidden),
            nn.LeakyReLU(0.1),
            nn.Linear(self.n_hidden,conv_channel3*self.size**2)
        )
            
    def forward(self,input):
        return torch.reshape(self.model(input),(-1,conv_channel3,self.size,self.size))

In [0]:
######################################################################
# Decoder
# ~~~~~~~~~
# Decoder Code
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder,self).__init__()
        self.channel = output_channel
        self.model=nn.Sequential(
            # layer 1
            nn.ConvTranspose2d(conv_channel3, conv_channel2, kernel_size=4, stride=4, bias=True),
            nn.BatchNorm2d(conv_channel2),
            nn.LeakyReLU(0.1),

            # layer 2
            nn.ConvTranspose2d(conv_channel2, conv_channel1, kernel_size=6, stride=4, bias=True),
            nn.BatchNorm2d(conv_channel1),
            nn.LeakyReLU(0.1),
#
            # layer 3
            nn.ConvTranspose2d(conv_channel1,self.channel, kernel_size=4, stride=4, bias=True),
            nn.BatchNorm2d(self.channel),
            nn.ReLU(True)

        )
    def forward(self,input):
        return self.model(input)

# Unet

In [0]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels):
        super(DoubleConv,self).__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super(Down,self).__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super(Up,self).__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)

        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.outconv=nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=1),
        nn.Sigmoid()
        )
        #self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        #return self.conv(x)
        return self.outconv(x)

In [0]:
class UNet(nn.Module):
    def __init__(self, n_channels=4, n_classes=1, bilinear=False):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        self.down4 = Down(512, 512)
        self.up1 = Up(1024, 256, bilinear)
        self.up2 = Up(512, 128, bilinear)
        self.up3 = Up(256, 64, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

# ShockNet


In [0]:
######################################################################
# ShockNet
# ~~~~~~~~~
# Shockt Code
class ShockNet(nn.Module):
    def __init__(self):
        super(ShockNet,self).__init__()
        self.encoder=Encoder()
        self.linear=Linear()
        self.decoder=Decoder()
        self.unet=UNet()

    def _forward(self,input,labels):

        # Encoder
        x=self.encoder(input)
        
        feature_1=x.shape[1]
        feature_2=x.shape[2]
        feature_3=x.shape[3]
        

        # Hidden
        x=torch.reshape(x,(-1,feature_1*feature_2*feature_3))
        x=torch.cat((x,labels),1)
        x=self.linear(x)

        # Decoder
        x=self.decoder(x)
        values=x

        # Shock Detection
        logits=self.unet(x)
        
        return values,logits

    def forward(self,input,labels):
        x=self._forward(input,labels)

        return x

In [46]:
shock_net=ShockNet().to(device)

# Handle multi-gpu if desired
if (device.type == 'cuda') and (ngpu > 1):
    shock_net= nn.DataParallel(shock_net, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
shock_net.apply(weights_init)

ShockNet(
  (encoder): Encoder(
    (model): Sequential(
      (0): Conv2d(1, 4, kernel_size=(4, 4), stride=(4, 4))
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.1)
      (3): Conv2d(4, 8, kernel_size=(4, 4), stride=(4, 4))
      (4): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): LeakyReLU(negative_slope=0.1)
      (6): Conv2d(8, 16, kernel_size=(4, 4), stride=(4, 4))
      (7): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): LeakyReLU(negative_slope=0.1)
    )
  )
  (linear): Linear(
    (model): Sequential(
      (0): Linear(in_features=146, out_features=10, bias=True)
      (1): LeakyReLU(negative_slope=0.1)
      (2): Linear(in_features=10, out_features=144, bias=True)
    )
  )
  (decoder): Decoder(
    (model): Sequential(
      (0): ConvTranspose2d(16, 8, kernel_size=(4, 4), stride=(4, 4))
      (1): BatchNorm2d(

# Tensorboard


In [0]:
#from torch.utils.tensorboard import SummaryWriter
#
## default `log_dir` is "runs" - we'll be more specific here
#writer = SummaryWriter('runs/')

In [0]:
#!pip install tensorboard==1.14.0

In [0]:
#!pip install tb-nightly

In [0]:
#writer.add_graph(shock_net,np.zeros((256,256)))
#writer.close()


In [0]:
#%load_ext tensorboard
#%tensorboard --logdir ./runs]

# loss function


In [51]:
# TODO  criteirion
def criterion(input)
optimzerG=optim.Adam(shock_net.parameters(),lr=lr,betas=(beta1,0.999))

SyntaxError: ignored

# Training Loop

In [52]:
# Training Loop

#for inputs, labels in train_dataloader:
#    #print(inputs)
#    print(inputs["pressure"].shape)
#    print(labels["Mach"][0])


# Lists to keep track of progress
G_losses = []
D_losses = []
shock_net.train()
print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(train_dataloader, 0):
        inputs,labels,outputs=data
        print(outputs[0].shape)

        ###########################
        inputs=inputs.to(device)
        labels=labels.to(device)
        inputs=inputs.type(torch.cuda.FloatTensor)
        labels=labels.type(torch.cuda.FloatTensor)
        ##########################
        ## Train with all-real batch
        shock_net.zero_grad()
        values,logits= shock_net(inputs,labels)
        ###########################

        #errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        #errD_real.backward()
        #D_x = output.mean().item()


Starting Training Loop...
torch.Size([4, 196, 196])
torch.Size([4, 196, 196])
torch.Size([4, 196, 196])
torch.Size([4, 196, 196])
torch.Size([4, 196, 196])
torch.Size([4, 196, 196])
torch.Size([4, 196, 196])
torch.Size([4, 196, 196])
torch.Size([4, 196, 196])
torch.Size([4, 196, 196])
